In [6]:
# Install required libraries in Colab (assuming already installed)
# !pip install folium geopandas shapely pandas

import pandas as pd
import folium
from folium import Choropleth
from folium.plugins import HeatMap, MarkerCluster
import geopandas as gpd
import numpy as np
from google.colab import files
from shapely.geometry import Point, MultiLineString, LineString
from shapely.ops import unary_union, nearest_points

# Load GeoJSON datasets
datasets = {
    "bike_lanes": gpd.read_file("hamilton_bike_lanes_osm.geojson"),
    "pedestrian_paths": gpd.read_file("hamilton_pedestrian_osm.geojson"),
    "streets_osm": gpd.read_file("hamilton_streets_osm.geojson"),
    "sidewalks": gpd.read_file("Road_Sidewalk.geojson"),
    "street_centerline": gpd.read_file("Street_Centreline.geojson"),
    "transit_areas": gpd.read_file("Transit_Service_Areas.geojson"),
    "bus_stops": gpd.read_file("HSR_Bus_Stops.geojson"),
    "bus_routes": gpd.read_file("HSR_Bus_Routes.geojson"),  # Assuming this is for bus routes
    "hospitals": gpd.read_file("Hospitals.geojson"),
    "schools": gpd.read_file("Educational_Institutions.geojson"),
    "wards": gpd.read_file("Ward_Boundaries.geojson"),
    "bikeways": gpd.read_file("Bikeways.geojson"),
    "bike_parking": gpd.read_file("Bike_Parking.geojson"),
    "bike_share_hubs": gpd.read_file("Hamilton_Bike_Share_Incorporated_Hubs.geojson")
}

# Clean datasets and ensure CRS is EPSG:4326 (WGS84)
for name, gdf in datasets.items():
    gdf.dropna(subset=['geometry'], inplace=True)
    gdf.to_crs(epsg=4326, inplace=True)

# Define city boundary
city_boundary = unary_union(datasets["wards"].geometry)

# Clip infrastructure to city boundary
for key in ["bike_lanes", "bikeways", "pedestrian_paths", "streets_osm", "sidewalks", "street_centerline", "bus_routes"]:
    datasets[key] = datasets[key].copy()
    datasets[key]["geometry"] = datasets[key].geometry.intersection(city_boundary)
    datasets[key] = datasets[key][~datasets[key].geometry.is_empty]

# Spatially join hospitals and schools with wards
for key in ["hospitals", "schools"]:
    datasets[key] = gpd.sjoin(datasets[key], datasets["wards"], how="left", predicate="intersects")
    datasets[key]["WARD"] = datasets[key]["WARD"].astype(str).replace("nan", np.nan)

# Center map on Hamilton, Ontario
hamilton_center = [43.2557, -79.8711]
m = folium.Map(location=hamilton_center, zoom_start=12, tiles="CartoDB positron")

# 1. Existing Infrastructure Layers
# Bike Lanes (Blue)
bike_layer = folium.FeatureGroup(name="Bike Lanes")
for _, row in datasets["bikeways"].iterrows():
    if isinstance(row.geometry, (LineString, MultiLineString)):
        coords = [(pt[1], pt[0]) for pt in (row.geometry.coords if isinstance(row.geometry, LineString) else [c for line in row.geometry.geoms for c in line.coords])]
        folium.PolyLine(locations=coords, color="blue", weight=3, popup=f"Ward: {row['WARD']}").add_to(bike_layer)
bike_layer.add_to(m)

# Sidewalks/Pedestrian Paths (Green)
ped_layer = folium.FeatureGroup(name="Pedestrian Paths")
for _, row in datasets["sidewalks"].iterrows():
    if isinstance(row.geometry, (LineString, MultiLineString)):
        coords = [(pt[1], pt[0]) for pt in (row.geometry.coords if isinstance(row.geometry, LineString) else [c for line in row.geometry.geoms for c in line.coords])]
        folium.PolyLine(locations=coords, color="green", weight=2, popup="Sidewalk").add_to(ped_layer)
ped_layer.add_to(m)

# Bus Routes (Red Lines)
bus_route_layer = folium.FeatureGroup(name="Bus Routes")
for _, row in datasets["bus_routes"].iterrows():
    if isinstance(row.geometry, (LineString, MultiLineString)):
        coords = [(pt[1], pt[0]) for pt in (row.geometry.coords if isinstance(row.geometry, LineString) else [c for line in row.geometry.geoms for c in line.coords])]
        folium.PolyLine(locations=coords, color="red", weight=3, popup=f"Bus Route<br>Ward: {row.get('WARD', 'Unknown')}").add_to(bus_route_layer)
bus_route_layer.add_to(m)

# Bus Stops (Red Markers)
bus_stop_layer = folium.FeatureGroup(name="Bus Stops")
for _, row in datasets["bus_stops"].iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=5, color="red", fill=True, fill_color="red", fill_opacity=0.6,
        popup=f"Bus Stop<br>Ward: {row.get('WARD', 'Unknown')}"
    ).add_to(bus_stop_layer)
bus_stop_layer.add_to(m)

# 2. Hospitals and Schools with Access Analysis
access_threshold = 400  # meters for all modes
infrastructure = {
    "bus": unary_union(datasets["bus_routes"].to_crs(epsg=32617).geometry.buffer(access_threshold).to_crs(epsg=4326)),
    "bike": unary_union(datasets["bikeways"].to_crs(epsg=32617).geometry.buffer(access_threshold).to_crs(epsg=4326)),
    "ped": unary_union(datasets["sidewalks"].to_crs(epsg=32617).geometry.buffer(access_threshold).to_crs(epsg=4326))
}

for key, icon, color in [("hospitals", "hospital", "purple"), ("schools", "school", "orange")]:
    gdf = datasets[key]
    gdf["bus_access"] = gdf.geometry.apply(lambda x: infrastructure["bus"].contains(x))
    gdf["bike_access"] = gdf.geometry.apply(lambda x: infrastructure["bike"].contains(x))
    gdf["ped_access"] = gdf.geometry.apply(lambda x: infrastructure["ped"].contains(x))

    cluster = MarkerCluster(name=f"{key.capitalize()}").add_to(m)
    for _, row in gdf.iterrows():
        access_score = sum([row["bus_access"], row["bike_access"], row["ped_access"]])
        marker_color = "green" if access_score == 3 else "yellow" if access_score > 0 else "red"
        folium.Marker(
            location=[row.geometry.y, row.geometry.x],
            popup=f"{key[:-1].capitalize()}: {row.get('NAME', 'Unknown')}<br>"
                  f"Bus Access: {row['bus_access']}<br>Bike Access: {row['bike_access']}<br>Ped Access: {row['ped_access']}<br>Ward: {row['WARD']}",
            icon=folium.Icon(color=marker_color, icon=icon, prefix="fa")
        ).add_to(cluster)

# 3. Development Opportunities
dev_layer = folium.FeatureGroup(name="Proposed Infrastructure")
for key in ["hospitals", "schools"]:
    uncovered = datasets[key][~(datasets[key]["bus_access"] & datasets[key]["bike_access"] & datasets[key]["ped_access"])]
    for _, row in uncovered.iterrows():
        loc_point = row.geometry
        for mode, mode_union, color in [("bus", infrastructure["bus"], "red"), ("bike", infrastructure["bike"], "blue"), ("ped", infrastructure["ped"], "green")]:
            if not row[f"{mode}_access"]:
                nearest_point = nearest_points(loc_point, mode_union)[1]
                new_route = LineString([loc_point, nearest_point]).intersection(city_boundary)
                if not new_route.is_empty and isinstance(new_route, (LineString, MultiLineString)):
                    coords = [(pt[1], pt[0]) for pt in (new_route.coords if isinstance(new_route, LineString) else [c for line in new_route.geoms for c in line.coords])]
                    folium.PolyLine(
                        locations=coords, color=color, weight=3, dash_array="5, 5",
                        popup=f"Proposed {mode.capitalize()} Route to {key[:-1].capitalize()}<br>Ward: {row['WARD']}"
                    ).add_to(dev_layer)
dev_layer.add_to(m)

# 4. Prepare Statistics for Buttons
# Hospital Access Stats
hospitals_full_access = datasets["hospitals"][datasets["hospitals"][["bus_access", "bike_access", "ped_access"]].all(axis=1)]
hospitals_no_access = datasets["hospitals"][~datasets["hospitals"][["bus_access", "bike_access", "ped_access"]].any(axis=1)]
hospitals_dev_needs = datasets["hospitals"][~datasets["hospitals"][["bus_access", "bike_access", "ped_access"]].all(axis=1)]

# School Access Stats
schools_full_access = datasets["schools"][datasets["schools"][["bus_access", "bike_access", "ped_access"]].all(axis=1)]
schools_no_access = datasets["schools"][~datasets["schools"][["bus_access", "bike_access", "ped_access"]].any(axis=1)]
schools_dev_needs = datasets["schools"][~datasets["schools"][["bus_access", "bike_access", "ped_access"]].all(axis=1)]

# Function to format stats for HTML
def format_list(gdf, name_col="NAME"):
    return "<br>".join([row[name_col] if pd.notna(row[name_col]) else "Unnamed" for _, row in gdf.iterrows()]) or "None"

def format_dev_needs(gdf, name_col="NAME"):
    result = []
    for _, row in gdf.iterrows():
        name = row[name_col] if pd.notna(row[name_col]) else "Unnamed"
        needs = []
        if not row["bus_access"]: needs.append("Bus")
        if not row["bike_access"]: needs.append("Bike")
        if not row["ped_access"]: needs.append("Pedestrian")
        result.append(f"{name}: {', '.join(needs)}")
    return "<br>".join(result) or "None"

# 5. Add Buttons with Statistics
button_html = """
<div style="position: fixed; top: 10px; left: 50px; z-index: 1000;">
    <button onclick="showStats('hospitals')">Hospital Access</button>
    <button onclick="showStats('schools')">School Access</button>
    <button onclick="showStats('dev')">Development Needs</button>
</div>
<div id="statsPopup" style="display: none; position: fixed; top: 50px; left: 50px; background: white; padding: 10px; border: 1px solid black; z-index: 1001; max-height: 80%; overflow-y: auto;">
    <h4 id="statsTitle"></h4>
    <p id="statsContent"></p>
    <button onclick="document.getElementById('statsPopup').style.display='none'">Close</button>
</div>
<script>
    function showStats(type) {
        var popup = document.getElementById('statsPopup');
        var title = document.getElementById('statsTitle');
        var content = document.getElementById('statsContent');
        popup.style.display = 'block';

        if (type === 'hospitals') {
            title.innerHTML = 'Hospital Access Statistics';
            content.innerHTML = `
                <b>Well-Served Hospitals (Bus, Bike, Walk):</b><br>${hospitals_full_access}<br><br>
                <b>Hospitals with No Access:</b><br>${hospitals_no_access}
            `;
        } else if (type === 'schools') {
            title.innerHTML = 'School Access Statistics';
            content.innerHTML = `
                <b>Well-Served Schools (Bus, Bike, Walk):</b><br>${schools_full_access}<br><br>
                <b>Schools with No Access:</b><br>${schools_no_access}
            `;
        } else if (type === 'dev') {
            title.innerHTML = 'Development Opportunities';
            content.innerHTML = `
                <b>Hospitals Needing Improvements:</b><br>${hospitals_dev_needs}<br><br>
                <b>Schools Needing Improvements:</b><br>${schools_dev_needs}
            `;
        }
    }
</script>
""".replace("${hospitals_full_access}", format_list(hospitals_full_access))\
   .replace("${hospitals_no_access}", format_list(hospitals_no_access))\
   .replace("${schools_full_access}", format_list(schools_full_access))\
   .replace("${schools_no_access}", format_list(schools_no_access))\
   .replace("${hospitals_dev_needs}", format_dev_needs(hospitals_dev_needs))\
   .replace("${schools_dev_needs}", format_dev_needs(schools_dev_needs))

m.get_root().html.add_child(folium.Element(button_html))

# 6. Add Layers Control and Title
folium.LayerControl().add_to(m)
m.get_root().html.add_child(folium.Element(
    "<h3 style='text-align:center;'>Hamilton Accessibility Analysis</h3>"
    "<p style='text-align:center;'>Access to Hospitals and Educational Institutions by Bus, Bike, and Walk (400m Buffer)</p>"
))

# Save and display
m.save("hamilton_accessibility.html")
from IPython.display import IFrame
IFrame(src="hamilton_accessibility.html", width=700, height=600)
files.download("hamilton_accessibility.html")

<ipython-input-6-14a1923f2009>:112: UserWarning: color argument of Icon should be one of: {'beige', 'blue', 'green', 'darkblue', 'lightgreen', 'gray', 'lightgray', 'darkred', 'white', 'red', 'cadetblue', 'lightred', 'pink', 'darkpurple', 'lightblue', 'black', 'orange', 'darkgreen', 'purple'}.
  icon=folium.Icon(color=marker_color, icon=icon, prefix="fa")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>